In [1]:
import pandas as pd
import numpy as np

In [122]:
data_10 = pd.read_csv('imputed_10_lags.csv')
data_10['Datetime'] = pd.to_datetime(data_10.loc[:,'Datetime'], infer_datetime_format=True)
data_10 = data_10.set_index('Datetime')

data_2 = pd.read_csv('imputed_2_lags.csv')
data_2['Datetime'] = pd.to_datetime(data_2.loc[:,'Datetime'], infer_datetime_format=True)
data_2 = data_2.set_index('Datetime')

data_original = pd.read_csv('data_original.csv')
data_original['Datetime'] = pd.to_datetime(data_original.loc[:,'Datetime'], infer_datetime_format=True)
data_original = data_original.set_index('Datetime')

In [17]:
data_10.describe().apply(lambda x: round(x, 3))

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,1527663.000,1527663.000,1527663.000,1527663.000,1527663.000,1527663.000,1527663.000
mean,1.071,0.126,241.336,4.531,1.082,1.168,6.675
std,1.000,0.113,2.927,4.194,5.993,5.376,8.445
min,0.076,-0.050,223.200,0.200,-16.845,-11.970,-2.607
25%,0.322,0.050,239.590,1.400,0.000,0.000,0.000
50%,0.652,0.102,241.336,2.800,0.000,0.000,1.000
75%,1.506,0.196,243.120,6.200,0.000,1.000,18.000
max,11.122,1.390,254.150,48.400,88.000,80.000,31.000


In [55]:
breakdown = data_original.describe(list(np.arange(0.1, 1.0, 0.05))).apply(lambda x: round(x, 3))
breakdown

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,1505619.000,1505619.000,1505619.000,1505619.000,1505619.000,1505619.000,1505619.000
mean,1.071,0.126,241.336,4.531,1.082,1.168,6.675
std,1.007,0.114,2.948,4.224,6.036,5.415,8.507
min,0.076,0.000,223.200,0.200,0.000,0.000,0.000
10%,0.234,0.000,237.770,1.000,0.000,0.000,0.000
15.0%,0.256,0.000,238.520,1.200,0.000,0.000,0.000
20.0%,0.292,0.000,239.090,1.200,0.000,0.000,0.000
25.0%,0.320,0.050,239.560,1.400,0.000,0.000,0.000
30.0%,0.342,0.060,239.960,1.600,0.000,0.000,0.000
35.0%,0.378,0.070,240.330,1.600,0.000,0.000,1.000


In [58]:

from sklearn.ensemble import IsolationForest

In [126]:
clf = IsolationForest(behaviour = 'new', max_samples=10000, random_state = 1, 
                      contamination= 'auto', verbose = 0, n_jobs = -1)
preds = clf.fit_predict(data_10)

In [138]:
not_null_original = data_original[~data_original.isnull().any(axis=1)]
clf_2 = IsolationForest(behaviour = 'new', max_samples=100000, random_state = 1, 
                      contamination= 'auto', verbose = 0, n_jobs = -1)
preds_2 = clf_2.fit_predict(not_null_original)

In [141]:
not_null_original.loc[preds_2 < 0,:].describe().\
                apply(lambda x: round(x, 4)).\
                to_csv("dash_app/data/eda/describe_original_outliers.csv")

In [142]:
not_null_original.loc[preds_2 > 0,:].describe().\
                apply(lambda x: round(x, 4)).\
                to_csv("dash_app/data/eda/describe_original_non_outliers.csv")

In [144]:
not_null_original.loc[preds_2 < 0,:] = np.nan

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [156]:
new_data = pd.concat([data_original[data_original.isnull().any(axis=1)], not_null_original])

In [168]:
import os
import pandas as pd
import re
import numpy as np
from joblib import dump, load
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pct_vars = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']
pct_vars_change = [i + "_pct_change" for i in pct_vars]
other = [i for i in new_data.columns if i not in pct_vars]
num_cols = new_data.columns

df_lag = new_data[pct_vars]
df_lag[pct_vars_change] = new_data[pct_vars].pct_change()
df_lag[other] = new_data[other]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [169]:
lag_cols = df_lag.columns
LAG = 3
for i in range(1, LAG):
    for col in lag_cols:
        df_lag[col + '__' + str(i)] = df_lag[col].shift(i)
        print(df_lag.shape)
df_lag.replace([np.inf, -np.inf], np.nan, inplace = True)


(1527663, 12)
(1527663, 13)
(1527663, 14)
(1527663, 15)
(1527663, 16)
(1527663, 17)
(1527663, 18)
(1527663, 19)
(1527663, 20)
(1527663, 21)
(1527663, 22)
(1527663, 23)
(1527663, 24)
(1527663, 25)
(1527663, 26)
(1527663, 27)
(1527663, 28)
(1527663, 29)
(1527663, 30)
(1527663, 31)
(1527663, 32)
(1527663, 33)


In [175]:
ii = IterativeImputer()
data_imputed = ii.fit_transform(df_lag)
from joblib import dump, load
dump(ii, 'iterative_imputer_2_lags_with_outliers.joblib')

imputed_2_lags = pd.DataFrame(data_imputed, columns = df_lag.columns, index = df_lag.index)
imputed_2_lags = imputed_2_lags.loc[:,num_cols]
imputed_2_lags.to_csv("imputed_2_lags_outliers_aware.csv")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/impute/_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [178]:
imputed_2_lags.describe().\
                apply(lambda x: round(x, 4)).\
            to_csv("dash_app/data/eda/describe_final.csv")